# NEP → GAA Budget Anomaly Detection

This notebook analyzes how proposed government budgets (NEP) change after approval (GAA).
It identifies unusual budget adjustments to support transparency and public understanding.

NEP = Proposed Budget  
GAA = Approved Budget


## Background

Government budgets go through a review process before final approval.
While proposed (NEP) and approved (GAA) budgets are public, understanding how and where changes happen is difficult.

This notebook focuses on:
- Identifying large or unusual changes
- Comparing regions fairly
- Avoiding false alarms
- Explaining results in simple language


## Dataset Overview

The dataset contains budget records with:
- Budget type (NEP or GAA)
- Fiscal year
- Budget amount
- Agency and region
- Budget classification and object codes

Only records from a selected agency are analyzed.


## Key Definitions

- NEP (National Expenditure Program): Proposed government budget
- GAA (General Appropriations Act): Final approved budget
- Budget Drift: Difference between NEP and GAA
- Anomaly: A budget adjustment that is unusually large or inconsistent with peers or history

Anomalies indicate items that may require review, not wrongdoing.


## Data Preparation

We separate NEP and GAA records and aggregate them to a comparable level
to ensure fair comparison.


In [3]:
import pandas as pd

df = pd.DataFrame()

fiscl_years = [2020, 2021, 2022, 2023, 2024, 2025, 2026]

for year in fiscl_years:
    df_year = pd.read_parquet(f'cleaned_budget_{year}.parquet')
    df = pd.concat([df, df_year], ignore_index=True)

nep = df[df['budget_type'] == 'NEP']
gaa = df[df['budget_type'] == 'GAA']

# Aggregate to comparable level
group_cols = [
    'fiscal_year',
    'department_code',
    'department_name',
    'full_agency_code',
    'agency_name',
    'region_code',
    'region_description',
    'uacs_object_code',
    'uacs_sub_object_name'
]

nep_agg = nep.groupby(group_cols)['budget_amount'].sum().reset_index()
gaa_agg = gaa.groupby(group_cols)['budget_amount'].sum().reset_index()

merged = nep_agg.merge(
    gaa_agg,
    on=group_cols,
    suffixes=('_nep', '_gaa')
)

## Measuring Budget Changes

We calculate:
- Absolute change (difference in amount)
- Percentage change (relative adjustment)

Small changes are common and expected.


In [5]:
# Calculate drift
merged['abs_change'] = merged['budget_amount_gaa'] - merged['budget_amount_nep']
merged['pct_change'] = merged['abs_change'] / merged['budget_amount_nep'].replace(0, 1)

## Interpreting Budget Adjustments

- NEP > GAA: Budget reduction
- NEP < GAA: Budget increase
- Minimal difference: Approved as proposed


In [6]:
def classify_adjustment(row):
    if row['pct_change'] > 0.05:
        return "Budget Increase"
    elif row['pct_change'] < -0.05:
        return "Budget Reduction"
    else:
        return "No Significant Change"

merged['adjustment_type'] = merged.apply(classify_adjustment, axis=1)

## Anomaly Detection Strategy (Summary)

This project uses multiple anomaly signals to reduce false positives.
A budget item is flagged only when it meets more than one condition:

- Meaningful budget change
- Unusual compared to peers
- Inconsistent with historical behavior

Final decisions are based on a combined anomaly score.


### Threshold Selection

Thresholds were chosen to balance sensitivity and false positives.
They are conservative by design and intended for review support, not enforcement.


In [7]:
merged['anomaly_threshold'] = (
    (merged['pct_change'].abs() > 0.3) |
    (merged['abs_change'].abs() > 50_000_000)
)

### Z-Score (Statistical Outliers)
Logic:

Compare each item to its peers (same year, same classification).

In [8]:
merged['z_score'] = merged.groupby(
    ['fiscal_year', 'uacs_object_code']
)['pct_change'].transform(
    lambda x: (x - x.mean()) / x.std(ddof=0)
)

merged['anomaly_zscore'] = merged['z_score'].abs() > 2

### Regional Comparison

Regions with no or minimal budget changes are treated as normal outcomes
of the approval process and are not flagged as anomalies.

In [9]:
merged['region_mean'] = merged.groupby(
    ['fiscal_year', 'uacs_object_code']
)['pct_change'].transform('mean')

merged['region_std'] = merged.groupby(
    ['fiscal_year', 'uacs_object_code']
)['pct_change'].transform('std')

MIN_CHANGE = 0.05

merged['region_anomaly'] = (
    (merged['pct_change'].abs() > MIN_CHANGE) &
    ((merged['pct_change'] - merged['region_mean']).abs() >
     2 * merged['region_std'])
)

### Historical Consistency Check

Logic:

Compare current drift against historical average drift.

In [10]:
merged['historical_mean'] = merged.groupby(
    ['full_agency_code', 'region_code', 'uacs_object_code']
)['pct_change'].transform('mean')

merged['historical_std'] = merged.groupby(
    ['full_agency_code', 'region_code', 'uacs_object_code']
)['pct_change'].transform('std')

merged['historical_anomaly'] = (
    (merged['pct_change'] - merged['historical_mean']).abs() >
    2 * merged['historical_std']
)

## Final Anomaly Scoring

In [11]:
merged['anomaly_score'] = (
    merged['anomaly_threshold'].astype(int) +
    merged['anomaly_zscore'].astype(int) +
    merged['historical_anomaly'].astype(int) +
    merged['region_anomaly'].astype(int)
)

merged['is_anomaly'] = merged['anomaly_score'] >= 2


Interpretation

Score 0–1: Normal

Score 2–3: Needs review

Score 4: High-risk anomaly

## Natural Language Explanation

In [15]:
def explain(row):
    reasons = []
    if row['anomaly_threshold']:
        reasons.append("large budget change")
    if row['anomaly_zscore']:
        reasons.append("statistically unusual compared to peers")
    if row['historical_anomaly']:
        reasons.append("inconsistent with historical trends")
    if row['region_anomaly']:
        reasons.append("differs from other regions")
    return '' if not reasons else "Flagged due to: " + ", ".join(reasons)

merged['explanation'] = merged.apply(explain, axis=1)

In [16]:
# Save the results
merged.to_parquet('detected_anomaly.parquet')

## Reviewing Sample Results

The following samples show:
- Random non-anomalous records
- Random detected anomalies

This helps validate that the detection logic behaves as expected.


In [21]:
# Sample some non-anomalies
merged[~merged['is_anomaly']].sample(5, random_state=42)

,fiscal_year,department_code,department_name,full_agency_code,agency_name,region_code,region_description,uacs_object_code,uacs_sub_object_name,budget_amount_nep,...,anomaly_zscore,region_mean,region_std,region_anomaly,historical_mean,historical_std,historical_anomaly,anomaly_score,is_anomaly,explanation
217230,2024,11,Department of Finance (DOF),11003,Bureau of Internal Revenue,02,Region II - Cagayan Valley,5010214001,Year-End Bonus - Civilian,15507.0,...,False,0.000114,0.002248,False,0.0,0.0,False,0,False,
38875,2020,20,Department of Social Welfare and Development (...,20001,Office of the Secretary,07,Region VII - Central Visayas,5010499015,Loyalty Award - Civilian,50.0,...,False,0.531667,11.234938,False,0.0,0.0,False,0,False,
77504,2021,15,Department of Justice (DOJ),15002,Bureau of Corrections,11,Region XI - Davao,5021305001,Machinery,544.0,...,False,-0.004701,0.051380,False,0.0,0.0,False,0,False,
99769,2021,38,Department of Transportation (DOTr),38001,Office of the Secretary,02,Region II - Cagayan Valley,5010304001,ECIP - Civilian,170.0,...,False,0.000876,0.025305,False,0.0,0.0,False,0,False,
39259,2020,20,Department of Social Welfare and Development (...,20001,Office of the Secretary,11,Region XI - Davao,5021305002,Office Equipment,250.0,...,False,-0.002606,0.065819,False,0.0,0.0,False,0,False,


In [22]:
# Sample some detected anomalies
merged[merged['is_anomaly']].sample(5, random_state=42)

,fiscal_year,department_code,department_name,full_agency_code,agency_name,region_code,region_description,uacs_object_code,uacs_sub_object_name,budget_amount_nep,...,anomaly_zscore,region_mean,region_std,region_anomaly,historical_mean,historical_std,historical_anomaly,anomaly_score,is_anomaly,explanation
27414,2020,14,Department of the Interior and Local Governmen...,14006,Philippine National Police,13,National Capital Region (NCR),5021503000,Insurance Expenses,68692.0,...,True,-0.003986,0.071424,True,0.069482,0.170195,True,4,True,"Flagged due to: large budget change, statistic..."
54241,2020,38,Department of Transportation (DOTr),38001,Office of the Secretary,16,Region XIII - CARAGA,5029901000,Advertising Expenses,160.0,...,False,2.200109,19.612446,False,-0.052083,0.127578,True,2,True,"Flagged due to: large budget change, inconsist..."
171219,2023,13,Department of Health (DOH),13001,Office of the Secretary,09,Region IX - Zamboanga Peninsula,5020301002,Office Supplies Expenses,28669.0,...,False,0.323357,6.733173,False,0.081389,0.199361,True,2,True,"Flagged due to: large budget change, inconsist..."
41749,2020,21,Department of Tourism (DOT),21001,Office of the Secretary,16,Region XIII - CARAGA,5029904000,Transportation and Delivery Expenses,100.0,...,False,0.246933,3.472248,False,0.131667,0.322516,True,2,True,"Flagged due to: large budget change, inconsist..."
46722,2020,24,"Department of Economy, Planning, and Developme...",24008,Philippine Statistics Authority,13,National Capital Region (NCR),5020301002,Office Supplies Expenses,149431.0,...,True,0.074238,0.430680,True,0.178455,0.437123,True,4,True,"Flagged due to: large budget change, statistic..."


## Limitations

- This analysis does not determine intent or correctness
- Results should be reviewed alongside official documents
- Some budget changes may be policy-driven or emergency-related


## Summary

This notebook demonstrates a transparent and explainable approach
to identifying unusual budget adjustments between NEP and GAA.

Future improvements include:
- Multi-year trend analysis
- Interactive dashboards
- Integration with Snowflake
